In [105]:
from pyspark.sql import SparkSession
import pandas as pd
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [6]:
spark = SparkSession.builder.appName("practice_session").getOrCreate()
spark

22/10/04 12:20:57 WARN Utils: Your hostname, adnan-System-Product-Name resolves to a loopback address: 127.0.1.1; using 192.168.1.109 instead (on interface enp2s0)
22/10/04 12:20:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/04 12:20:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [257]:
from pyspark.sql.functions import pandas_udf,PandasUDFType
from pyspark.sql.types import *

@pandas_udf("int")
def calAvg(series:pd.Series)->float:
    return series.mean()


In [197]:
df = spark.read.csv("practice_data.csv",header=True,inferSchema=True)
df.na.fill({
    "age":df.select(calAvg(df.age)).collect()[0][0],
    "name":"unknown",
    "experience":0,
    "salary":df.select(calAvg(df.age)).collect()[0][0],
})

Name,age,experience,salary
Krish,31,10,30000
Sudhanshu,30,8,25000
Sunny,29,4,20000
Paul,24,3,20000
Harsha,21,1,15000
Shubham,23,2,18000
Mahesh,28,0,40000
unknown,34,10,38000
unknown,36,0,28


In [311]:
df2 = spark.createDataFrame([
    Row(Name="Krish",Departments="Data Science",Salary=10000),
    Row(Name="Krish",Departments="IOT",Salary=5000),
    Row(Name="Krish",Departments="Big Data",Salary=4000),
    Row(Name="Sudhanshu",Departments="IOT",Salary=20000),
    Row(Name="Sudhanshu",Departments="Data Science",Salary=10000),
    Row(Name="Sudhanshu",Departments="Big Data",Salary=5000),
    Row(Name="Sunny",Departments="Data Science",Salary=10000),
    Row(Name="Sunny",Departments="Big Data",Salary=2000),
    Row(Name="Mahesh",Departments="Data Science",Salary=4000),
    Row(Name="Mahesh",Departments="Big Data",Salary=3000)
    ])

In [312]:
min(df2.groupBy("Name").sum().collect(),key=lambda x:x[1])

Row(Name='Mahesh', sum(Salary)=7000)

In [313]:
df2.groupBy("Name").sum("Salary")

Name,sum(Salary)
Krish,19000
Sudhanshu,35000
Sunny,12000
Mahesh,7000


In [314]:
def calAvg(frame):
    return frame.assign(Salary=frame.Salary-frame.Salary.mean())
    
df2.groupBy("Name").applyInPandas(calAvg,schema=df2.schema)

Name,Departments,Salary
Krish,Data Science,3666
Krish,IOT,-1333
Krish,Big Data,-2333
Mahesh,Data Science,500
Mahesh,Big Data,-500
Sudhanshu,IOT,8333
Sudhanshu,Data Science,-1666
Sudhanshu,Big Data,-6666
Sunny,Data Science,4000
Sunny,Big Data,-4000


In [326]:
df = df.na.fill({
    "age":df.summary("mean").collect()[0].asDict()["age"],
    "name":"Unknown",
    "experience":0,
    "salary":df.summary("min").collect()[0].asDict()["salary"]
})

In [327]:
df

Name,age,experience,salary
Krish,31,10,30000
Sudhanshu,30,8,25000
Sunny,29,4,20000
Paul,24,3,20000
Harsha,21,1,15000
Shubham,23,2,18000
Mahesh,28,0,40000
Unknown,34,10,38000
Unknown,36,0,15000


In [334]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["age","experience"],outputCol="Independent_Feature")
indFeat = assembler.transform(df)
final_data = indFeat.select(["Independent_Feature","salary"])
train,test = final_data.randomSplit([0.7,0.3])

In [338]:
from pyspark.ml.regression import LinearRegression

regression= LinearRegression(featuresCol="Independent_Feature",labelCol="salary")
regression.fit(train)

22/10/04 19:37:11 WARN Instrumentation: [5f7c950e] regParam is zero, which might cause numerical instability and overfitting.


LinearRegressionModel: uid=LinearRegression_7ab103419288, numFeatures=2

In [340]:
regression.fit(train).evaluate(test).predictions.show()

22/10/04 19:38:42 WARN Instrumentation: [f6b6c9c8] regParam is zero, which might cause numerical instability and overfitting.
+-------------------+------+------------------+
|Independent_Feature|salary|        prediction|
+-------------------+------+------------------+
|         [21.0,1.0]| 15000|25420.650730411795|
|         [23.0,2.0]| 18000|25830.677290836742|
+-------------------+------+------------------+

22/10/04 20:08:47 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/blockmgr-2e0220e7-4da3-40e1-a4de-ef5bc30c659c. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/blockmgr-2e0220e7-4da3-40e1-a4de-ef5bc30c659c
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52200)
Traceback (most recent call last):
  File "/home/slowgamer/anaconda3/envs/pyspark_env/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/slowgamer/anaconda3/envs/pyspark_env/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/slowgamer/anaconda3/envs/pyspark_env/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/slowgamer/anaconda3/envs/pyspark_env/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/slowgamer/anaconda3/envs/pyspark_env/lib/python3.10/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/home/slowgamer/anaconda3/envs/pyspark_env/lib/python3.10/site-